In [1]:
import xmltodict as xd
import pandas as pd
import os, glob
import mio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def get_model_list(kml_path:str)->pd.DataFrame:
    " read kml file, get anchor coordinates for every model"
    mio.check_types(get_model_list, locals())
    with open(kml_path) as fin:
        s = fin.read()
    kmld =  xd.parse(s)
    rows = []
    for pm in kmld['kml']['Document']['Placemark'][0:]:
        model = pm['Model']
        row = {}
        row['model'] = os.path.basename(model['Link']['href'])
        sLon = model['Location']['longitude']
        sLat = model['Location']['latitude']
        row['swiss_lon'] = float(sLon)
        row['swiss_lat'] = float(sLat)
        row['swiss_x'], row['swiss_y'] = mio.wgs_swiss(sLon, sLat)
        row['geometry'] = Point(row['swiss_x'], row['swiss_y'])
        rows.append(row)
    df = pd.DataFrame(rows)
    df = df.set_index('model')
    return df

In [3]:
def get_point_lists(dae_file:str) -> dict:
    " get point and vertex list for one model"
    with open(dae_file) as fin:
        s = fin.read()
    d = xd.parse(s)
    
    geometries = d['COLLADA']['library_geometries']['geometry']
    
    res = {}
    for geo in geometries:
        geo_id = geo['@id']

        # float list
        float_list = geo['mesh']['source']['float_array']['#text'].split()
        float_list = [float(f) for f in float_list]

        # vertex list
        vert_list = geo['mesh']['triangles']['p'].split()
        vert_list = [int(n) for n in vert_list]
        # write result
        res[geo_id] = (float_list, vert_list)
    return res

In [4]:
def append_geos(dic:dict):
    " combine the v_lists and f_lists in the dictinary to one "
    v_list = []
    f_list = []
    for key in dic:
        offset = len(v_list) // 3
        v_list = v_list + dic[key][0] 
        f_list = f_list + [n+offset for n in dic[key][1]]
    return v_list, f_list

In [5]:
def write_obj(v_list:list, f_list:list, lon:float, lat:float, dest:str):
    " write v_list and f_list to an obj text 3d file"
    fout = open(dest, 'w')
    fout.write(f'g {lon} {lat}\n')
    
    va = np.array(v_list)
    va = va.reshape(len(v_list)//3, 3)
    for row in va:
        s = f"v {row[0]} {row[1]} {row[2]}\n"
        fout.write(s)
        
    fa = np.array(f_list)
    fa = fa.reshape(len(f_list)//3, 3)
    for row in fa:
        s = f"f {row[0]+1} {row[1]+1} {row[2]+1}\n"
        fout.write(s)       
    
    fout.close()

In [6]:
def make_obj(dae_path:str, dest:str, lon:float, lat:float):
    " full proecess of converting .dae to .obj file"
    dic = get_point_lists(dae_path)
    v_list, f_list = append_geos(dic)
    write_obj(v_list, f_list, lon, lat, dest)

In [7]:
def convert_kml(kml_path:str, dest_folder:str):
    prefix = mio.file_title(kml_path) + '_'
    mod_list = get_model_list(kml_path)
    i = 0
    for ind, row in mod_list.iterrows():
        i += 1
        dae_path = os.path.join(os.path.dirname(kml_path), 'models', ind)
        dest = dest_folder + "/" + prefix + mio.file_title(dae_path) + '.obj'
        try:
            make_obj(dae_path, dest, row.swiss_lon, row.swiss_lat)
        except:
            print('error in conver_kml()', kml_path)
        
    return i

In [8]:
def convert_folder(title):
    dirs = glob.glob(rf"C:\Buildings2.0\KML\{title}-*")
    dest_folder = f'json/{title}'
    print(dest_folder, end=' ')
    if not os.path.isdir(dest_folder):
        os.mkdir(dest_folder)
    obj_count = 0
    for i, dr in enumerate(dirs):
        mio.show_perc(i, len(dirs), 1)
        ft = mio.file_title(dr) + '.kml'
        kml_path = os.path.join(dr, ft)
        obj_count = obj_count + convert_kml(kml_path, dest_folder)
    print(obj_count, 'files')

In [9]:
titles = glob.glob(rf"C:\Buildings2.0\KML\1091*")
titles = [mio.file_title(s)[0:4] for s in titles]
titles = list(set(titles))
titles.sort()
len(titles)

1

In [10]:
for title in titles:
    convert_folder(title)

json/1091 0.0% 6.25% 12.5% 18.75% error in conver_kml() C:\Buildings2.0\KML\1091-14\1091-14.kml
25.0% 31.25% error in conver_kml() C:\Buildings2.0\KML\1091-22\1091-22.kml
error in conver_kml() C:\Buildings2.0\KML\1091-22\1091-22.kml
37.5% error in conver_kml() C:\Buildings2.0\KML\1091-23\1091-23.kml
43.75% 50.0% 56.25% error in conver_kml() C:\Buildings2.0\KML\1091-32\1091-32.kml
62.5% 68.75% error in conver_kml() C:\Buildings2.0\KML\1091-34\1091-34.kml
75.0% error in conver_kml() C:\Buildings2.0\KML\1091-41\1091-41.kml
error in conver_kml() C:\Buildings2.0\KML\1091-41\1091-41.kml
81.25% 87.5% 93.75% 95633 files
